In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import h5py
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from sklearn.model_selection import KFold,cross_val_score
kfold=KFold(5,True,3)
cvscores = []


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load training dataset
# Name for the column  or Features Map
columns_to_named = ["frameNumber", "frameTime", "frameLen", "ethSrc", "ethDst",
       "ipSrc", "ipDst", "ipProto", "ipLen", "tcpLen", "tcpSrcport",
       "tcpDstport", "Value", "normality"]
    
    # Read the Dataset and Rename the Column
df = pd.read_csv("/content/drive/My Drive/Preprocessed_data.csv", header=0,names=columns_to_named)

col_norm = ['frameNumber', 'frameTime', 'frameLen', 'ethSrc', 'ethDst',
       'ipSrc', 'ipDst', 'ipProto', 'ipLen', 'tcpLen', 'tcpSrcport',
       'tcpDstport', 'Value']
    
X = df[col_norm]
Y = df['normality']    

In [ ]:
df.head()

,frameNumber,frameTime,frameLen,ethSrc,ethDst,ipSrc,ipDst,ipProto,ipLen,tcpLen,tcpSrcport,tcpDstport,Value,normality
0,1,1.237230e+14,54,8.797200e+13,1.672760e+14,1.921680e+08,1.921680e+09,6,40.0,0.0,49279.0,80,-99.0,0
1,2,1.237230e+14,62,8.797200e+13,1.672760e+14,1.921680e+08,1.921680e+09,6,48.0,0.0,56521.0,80,-99.0,0
2,3,1.237230e+14,62,1.672760e+14,8.797200e+13,1.921680e+09,1.921680e+08,6,48.0,0.0,80.0,56521,-99.0,0
3,4,1.237230e+14,54,1.672760e+14,8.797200e+13,1.921680e+09,1.921680e+08,6,40.0,0.0,80.0,49279,-99.0,0
4,5,1.237230e+14,54,8.797200e+13,1.672760e+14,1.921680e+08,1.921680e+09,6,40.0,0.0,56521.0,80,-99.0,0


In [ ]:
from sklearn.preprocessing import label_binarize

y = label_binarize(Y, classes=[0, 1, 2, 3, 4, 5])
n_classes = y.shape[1]
n_classes

6

In [ ]:
target_names = ['Normal', 'WrongSetup', 'DDoS', 'Probing', 'Scan', 'MITM']

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

for train, test in kfold.split(X,Y):

    X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    x_train = np.array(X_train)
    x_test = np.array(X_test)
    y_train1 = np.array(y_train)
    y_test1 = np.array(y_test)

    y_train= to_categorical(y_train1)
    y_test= to_categorical(y_test1)

    model = Sequential()
    model.add(LSTM(4, input_dim=13))  # try using a GRU instead, for fun
    model.add(Dropout(0.6))
    model.add(Dense(6))
    model.add(Activation('softmax'))
    #model.summary()

    model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy','mae'])
    history = model.fit(X_train, y_train, epochs = 10, batch_size=64, verbose=1,shuffle=False)
    scores = model.evaluate(X_test, y_test, verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/10
5968/5968 [==============================] - 10s 2ms/step - loss: 0.9976 - accuracy: 0.5263 - mae: 0.1851
Epoch 2/10
5968/5968 [==============================] - 9s 2ms/step - loss: 0.8401 - accuracy: 0.5962 - mae: 0.1572
Epoch 3/10
5968/5968 [==============================] - 9s 2ms/step - loss: 0.8072 - accuracy: 0.6456 - mae: 0.1495
Epoch 4/10
5968/5968 [==============================] - 10s 2ms/step - loss: 0.7852 - accuracy: 0.6573 - mae: 0.1438
Epoch 5/10
5968/5968 [==============================] - 9s 2ms/step - loss: 0.7741 - accuracy: 0.6569 - mae: 0.1410
Epoch 6/10
5968/5968 [==============================] - 9s 2ms/step - loss: 0.7627 - accuracy: 0.6597 - mae: 0.1384
Epoch 7/10
5968/5968 [==============================] - 9s 2ms/step - loss: 0.7496 - accuracy: 0.6706 - mae: 0.1361
Epoch 8/10
5968/5968 [==============================] - 9s 2ms/step - loss: 0.7345 - accuracy: 0.7001 - mae: 0.1332
Epoch 9/10
5968/5968 [==============================] - 9s 2ms/step - 

In [ ]:
### train_accuracy
import sklearn.metrics as metrics
expected = y_train
predicted = model.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)

print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      Normal       1.00      0.98      0.99     63266
  WrongSetup       0.99      1.00      0.99     65753
        DDoS       1.00      1.00      1.00     63212
     Probing       1.00      0.99      1.00     63179
        Scan       1.00      0.99      0.99     63278
        MITM       0.99      1.00      0.99     63252

   micro avg       1.00      0.99      0.99    381940
   macro avg       1.00      0.99      0.99    381940
weighted avg       1.00      0.99      0.99    381940
 samples avg       0.99      0.99      0.99    381940

Accuracy: 0.9936508352097188
Precision: 0.9950900177960984
Recall: 0.9936508352097188
f1-score: 0.9943348109286005
ROC_AUC: 0.9983805142281816


In [ ]:
###for test accuracy
expected = y_test
predicted = model.predict(X_test)
data = classification_report(expected, np.round(predicted), target_names=target_names)
print(data)

print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      Normal       1.00      0.98      0.99     15769
  WrongSetup       0.99      1.00      0.99     16532
        DDoS       1.00      1.00      1.00     15808
     Probing       1.00      0.99      1.00     15823
        Scan       1.00      0.99      0.99     15774
        MITM       0.99      1.00      0.99     15780

   micro avg       1.00      0.99      0.99     95486
   macro avg       1.00      0.99      0.99     95486
weighted avg       1.00      0.99      0.99     95486
 samples avg       0.99      0.99      0.99     95486

Accuracy: 0.9937163563244874
Precision: 0.9953216038945082
Recall: 0.9937163563244874
f1-score: 0.9944857548766625
ROC_AUC: 0.9984061239515216
